# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
# load data from database
from sqlalchemy import create_engine

engine = create_engine('sqlite:///DisasterResponse.db')
connection = engine.connect()
df = pd.read_sql_table(engine.table_names()[0], connection)

X = df['message']
Y = df.loc[:, df.columns != 'message']

In [3]:
Y.head(5)

,index,id,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,12,facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
### index, id does not seem to have any factual implication 
### original seems to be replica of message but in other languages
### but what about genre? 
Y.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

<p>The value indicates the source where does the messaege derives from
It may seem to matter how importance/priority we treat the messages 
but if we treat each message with equal importance, then this message
may not seem important(the criteria for project seems to be classify responses to particular relief organization for handle</p>



In [5]:
### drop some columns that does not seem useful
Y = Y.drop(columns=['index', 'id', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [6]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

text = "Dr. Smith graduated from the University of Washington. He later started an analytics firm called Lux, which catered to enterprise customers."
print(tokenize(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['dr', 'smith', 'graduated', 'university', 'washington', 'later', 'started', 'analytics', 'firm', 'called', 'lux', 'catered', 'enterprise', 'customer']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

#### To build a classifier to separate them into class
#### We try to choose from ensemble type of learning classifier, trees

# def build_model():
#pipeline = Pipeline([
#    ('vect', CountVectorizer(tokenizer=tokenize)),
#    ('tfidf', TfidfTransformer()),
#    ('clf', MultiOutputClassifier(KNeighborsClassifier())),
#])
    
    
pipeline = Pipeline([
        ('feat', FeatureUnion([
            ('pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ]))            
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])



#pipeline = Pipeline(
#                    [('count_vectorizer', CountVectorizer(tokenizer = tokenize)),
#                     ('tfidf_transformer', TfidfTransformer()),
#                     ('rf_classifier', RandomForestClassifier())
#                     ])
#     return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# model = build_model()
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('feat', FeatureUnion(n_jobs=1,
       transformer_list=[('pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
### get parameters we can tune the pipeline for
pipeline.get_params()



Reference:
<ol>
    
<li>https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff</li>

<li>https://medium.datadriveninvestor.com/a-survey-of-evaluation-metrics-for-multilabel-classification-bb16e8cd41cd</li>
</ol>

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'feat__pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'feat__pipeline__vect__max_df': (0.5, 0.75, 1.0),
    'feat__pipeline__vect__max_features': (None, 5000, 10000),
    'feat__pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
    'clf__estimator__n_estimators': [10, 20, 40]
}

cv =  GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train,y_train)

y_pred = pipeline.predict(X_test)

for category in y_test:
    print('Category: {}'.format(category))
    idx = y_test.columns.get_loc(category)
    print(classification_report(y_test[category], y_pred[:,idx]))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
y_test.sum(axis=0)



### 9. Export your model as a pickle file

In [ ]:
import pickle

Pkl_Filename = "classifier.pkl"  



with open(Pkl_Filename, 'wb') as file:  
    pickle.dumps(pipeline,file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.